In [1]:
import igl
import math
import scipy as sp
import numpy as np
import meshplot as mp
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import os
root_folder = os.getcwd()

In [2]:
v_init, f = igl.read_triangle_mesh(os.path.join(root_folder, "oblate.off"))
### Parameters for running computation
global Kb
global Kv
global Ka
global gamma
global KbT
global delT
global Kal
Kal=1
gamma=0.01
Ka=20
Kv=10
Kb=0.01
H0=0
Volume_t= 0.6* 3.14 * 4 / 3
Area_t=4*3.14
KbT=0.01
dt=0.02
sigma=np.sqrt(2*gamma*(KbT/dt))
hdt=0.5*dt
iterations = 9999
outfrequency = 100
tolerance = 0.0001
maxError = 10000

In [3]:
###Energy Calculations Area+Bending+Volume
def Energy_area(v,f,Area_t):
    Area_new=cal_areatot(v,f)
    Energy_Area=Ka*((Area_new-Area_t)**2)/Area_t
    return Energy_Area
def Energy_volume(v,f,Volume_t):
    volume_new=cal_volumetot(v,f)
    Energy_volume=Kv*((volume_new-Volume_t)**2)/Volume_t
    return Energy_volume
def Energy_bending(v,f,H0):
    npv = igl.per_vertex_normals(v, f)
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.sparse.diags(1 / m.diagonal())
    area_voronoi=m.diagonal()
    l = igl.cotmatrix(v, f) ###laplacian-operator
    Hn = -minv.dot(l.dot(v))/2
    H_mean = np.linalg.norm(Hn, axis=1)
    sign_H = np.sign(np.sum(Hn*npv, axis=1))
    H_mean_signed = H_mean*sign_H
    #print(max(h_mean))
    #print(min(h_mean))
    Eb = 2*Kb*((H_mean_signed-H0)**2)*area_voronoi
    total_EB = np.sum(Eb)
    return total_EB
def Kinetic_energy(v,f,velocity):
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    area_voronoi=m.diagonal()
    KE = 0.5*np.linalg.norm(velocity, axis=1)**2*area_voronoi
    total_KE = np.sum(KE)
    return total_KE    

In [4]:
def adjacent_face(v,f):
    df=pd.DataFrame(f,columns=list('ABC'))
    row_numbers=[]
    for i in range(len(v)):
        row_numbers.append((df.index[(df['A'] == i)|(df['B'] == i) | (df['C'] == i)].tolist()))
    return row_numbers

def cal_areatot(v,f):
    dbl_area = igl.doublearea(v, f)    
    Areatot = np.sum(dbl_area)/2
    return Areatot

def cal_volumetot(v,f):
    Volumetot = 0
    for i in range(len(f)):
        sum=0
        p0x=v[f[i][0]][0]
        p0y=v[f[i][0]][1]
        p0z=v[f[i][0]][2]
        p1x=v[f[i][1]][0]
        p1y=v[f[i][1]][1]
        p1z=v[f[i][1]][2]
        p2x=v[f[i][2]][0]
        p2y=v[f[i][2]][1]
        p2z=v[f[i][2]][2]
        v321= p2x*p1y*p0z
        v231= p1x*p2y*p0z
        v312= p2x*p0y*p1z
        v132= p0x*p2y*p1z
        v213= p1x*p0y*p2z
        v123= p0x*p1y*p2z
        sum=(-v321+ v231+ v312-v132-v213+ v123) / 6.0
        #print(sum)
        Volumetot+=sum
    return Volumetot

def areaGrad(v,f):
    #n=igl.per_vertex_normals(v,f) ## not using per_vertex_normals for areaGrad direction
    l = igl.cotmatrix(v, f) ###laplacian-operator
    ag = -l.dot(v)
    return ag

def volGrad(v,f):
    npv = igl.per_vertex_normals(v, f)
    face_normal=igl.per_face_normals(v,f,npv)
    dbl_area = igl.doublearea(v, f)
    adjacent_vertices=igl.adjacency_list(f)
    adjacent_faces=adjacent_face(v,f)
    volumegrad=[]
    for i in range(len(v)):
        vol_ij=0
        for j in range(len(adjacent_faces[i])):
            k=adjacent_faces[i][j]
            Area=dbl_area[k]/2
            FaceNorm=face_normal[k]
            vol_ij += (1/3)*Area*FaceNorm
        volumegrad.append(vol_ij)
    return np.array(volumegrad)

adjacent_faces=adjacent_face(v,f)
k=adjacent_faces[0][2]
n = igl.per_vertex_normals(v, f)
face_normal=igl.per_face_normals(v,f,n)
face_normal[k]

def fun_volgrad(v,f):
    n = igl.per_vertex_normals(v, f)
    #vector=np.array([1,1,1])
    #norm=vector/np.linalg.norm(vector)

    face_normal=igl.per_face_normals(v,f,n)
    dbl_area = igl.doublearea(v, f)/2
    volume_grad=face_normal*dbl_area[:,None]
    
    return volume_grad/3

In [5]:
###Force from bending
def fun_ForceDensity(v,f):
    npv = igl.per_vertex_normals(v, f)
    K = igl.gaussian_curvature(v, f)
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.sparse.diags(1 / m.diagonal())
    area_voronoi=m.diagonal()
    l = igl.cotmatrix(v, f) ###laplacian-operator
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.sparse.diags(1 / m.diagonal())
    Hn = -minv.dot(l.dot(v))/2
    H_mean = np.linalg.norm(Hn, axis=1)
    sign_H = np.sign(np.sum(Hn*npv, axis=1))
    #if (min(sign_H)<0):
    #    print('H_mean changes sign')
    H_mean_signed = H_mean*sign_H
    Lap_H = minv.dot(l.dot(H_mean_signed-H0))
    first_term = 2*(H_mean_signed-H0)*(H_mean_signed**2 + H0*H_mean_signed - K)
    totalforce = first_term + Lap_H
    #normal_v=n/np.linalg.norm(n)
    ### Force Density and Nodal Force
    #Force_density=[] # Force_Density
    #Force_Nodal=[]

    #for i in range (len(v)):
    Force_density = 2*Kb*npv*totalforce[:,None]
        #Force_Nodal.append(Force_density[i]*area_voronoi[i])
        #print(Force_density[i]) 
        #print(Force_Nodal[i])
        
    return Force_density

In [6]:
###Force from Area Constraints
def Force_Area(Area_t,grad_Area,Area_new):
    #grad_Area=volgrad(v,f)
    #Area_old=cal_volume(v,f)
    #Area_new=cal_Area(pos_new,f)
    Force_Area=-2*(Ka)*((Area_new-Area_t)/Area_t)*grad_Area
    return Force_Area

In [7]:
###Force from Volume Area Constraints
def Force_Volume(Volume_t,grad_Volume,volume_new):
    #grad_Volume=volgrad(v,f)
    #Volume_old=cal_volume(v,f)
    #Volume_new=cal_volume(pos_new,f)

    Force_Volume=-2*(Kv)*((volume_new-Volume_t)/Volume_t)*grad_Volume ## Volume constraint is not the same as in mem3dg
    return Force_Volume

In [8]:
###Dissipative Force
def fun_FD(v,f,velocity):
    adjacent_vertices=igl.adjacency_list(f)
    FDij_nodes=[]
    for i in range(len(v)):
        FD_ij=0
        for j in range(len(adjacent_vertices[i])):
            k=adjacent_vertices[i][j]
            vij=velocity[i]-velocity[k]
            rij=v[i]-v[k]
            #rij_distance=math.sqrt((rij[0][0]**2)+(rij[0][1]**2)+(rij[0][2]**2))
            rij_norm = rij/np.linalg.norm(rij)
            FD_ij=FD_ij+(-((gamma*np.dot(vij, rij)*rij_norm)))
        FDij_nodes.append(FD_ij) 
    return FDij_nodes

In [9]:
###Dissipative Force
def fun_FD2(v,f,velocity):
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    area_voronoi=m.diagonal()
    FD_nodes = -gamma*velocity/area_voronoi[:,None]
    return FD_nodes

In [10]:
####Random Forces
def fun_FR(v,f): 
    edges, fe, ef = igl.edge_topology(v, f)
    Fr=np.zeros((len(v),3))
    for e in range(len(edges)): 
        rij=v[edges[e][0]]-v[edges[e][1]]
        rij_norm = rij/np.linalg.norm(rij)
        gaussian=np.random.normal(0, 1, 1)
        Force=gaussian*rij_norm*sigma
        Fr[edges[e][0]]+=Force
        Fr[edges[e][1]]-=Force  
    return Fr

In [11]:
####Random Forces
def fun_FR2(v,f): 
    gaussian = np.random.normal(0, 1, 3*len(v))
    FR = sigma*gaussian.reshape(len(v),3)
    return FR

In [12]:
def Total_Force(FB,FA,FV,FD):
    Total_force=(FB+FA+FV+FD)
    return Total_force

In [13]:
#Volume_old=cal_volume(v,f)
#Area_t=cal_Area(v,f)
grad_Area=areaGrad(v_init,f)
grad_Volume=volGrad(v_init,f)
Area_new=cal_areatot(v_init,f)
print('initial area = ',Area_new)
Volume_new=cal_volumetot(v_init,f)
print('initial volume = ', Volume_new)

initial area =  12.478260465930916
initial volume =  4.024773179610609


grad_A=areaGrad(v,f)

grad_Volume=volgrad(v,f)
grad_Volume=np.array(grad_Volume)
grad_Volume.shape

In [ ]:
#%%time
v_init, f = igl.read_triangle_mesh(os.path.join(root_folder, "oblate.off"))### Can be cahnged for Prolate

###Velocity Verlet Main 

#Setup
v=v_init
vel=np.zeros((len(v_init),3))
FD=fun_FD2(v_init,f,vel)
FB=fun_ForceDensity(v_init,f)
FA=Force_Area(Area_t,grad_Area,Area_new)
FV=Force_Volume(Volume_t,grad_Volume,Volume_new)
#FR=fun_FR2(v,f)
TF=Total_Force(FB,FA,FV,FD)
### Calculation of Energy
totalEnergy=[]
EnergyArea=[]
EnergyVolume=[]
EnergyBending=[]
KineticEnergy=[]
#df=open('iter=0.01_100,gamma=100','w')
#df.write('Total Energy of the System, dt=0.01\n')
#new_file = open('FD.txt','w')
for i in range(iterations):
    #Initial integration
    vel += 0.5*TF*dt
    v += vel*dt
   
    #Force calculation
    Area_current=cal_areatot(v,f)
    grad_Area_new=areaGrad(v,f)
    Volume_current=cal_volumetot(v,f)
    grad_Volume_new=volGrad(v,f)

    ###Update forces here
    #FR=fun_FR2(v,f)
    FD=fun_FD2(v,f,vel)
    FB=fun_ForceDensity(v,f) ##bending_force
    FA=Force_Area(Area_t,grad_Area_new,Area_current)
    #print(FAL)
    FV=Force_Volume(Volume_t,grad_Volume_new,Volume_current)
    TF=Total_Force(FB,FA,FV,FD)

    #     if i>1300 and i%5==0:
    #         ax1 = plt.figure().add_subplot(projection='3d')
    #         ax1.quiver(v[:,0], v[:,1],v[:,2], FA[:,0], FA[:,1], FA[:,2], length=0.2, normalize=True,color='g')
    #         filename = '/home/BU/dredwan1/Velocity Verlet/Gamma=100_Oblate/FA-images/'
    #         plt.savefig(filename + 'TEST_' + str(i) + '.png',dpi=600)
    #         ax2 = plt.figure().add_subplot(projection='3d')
    #         ax2.quiver(v[:,0], v[:,1],v[:,2], FV[:,0], FV[:,1], FV[:,2], length=0.2, normalize=True,color='r')
    #         filename = '/home/BU/dredwan1/Velocity Verlet/Gamma=100_Oblate/FV-images/'
    #         plt.savefig(filename + 'TEST_' + str(i) + '.png',dpi=600)
    #         ax3= plt.figure().add_subplot(projection='3d')
    #         ax3.quiver(v[:,0], v[:,1],v[:,2], FB[:,0], FB[:,1], FB[:,2], length=0.2, normalize=True,color='b')
    #         filename = '/home/BU/dredwan1/Velocity Verlet/Gamma=100_Oblate/FB-images/'
    #         plt.savefig(filename + 'TEST_' + str(i) + '.png',dpi=600)
    #         ax4= plt.figure().add_subplot(projection='3d')
    #         ax4.quiver(v[:,0], v[:,1],v[:,2], FD[:,0], FD[:,1], FA[:,2], length=0.2, normalize=True,color='y')
    #         filename = '/home/BU/dredwan1/Velocity Verlet/Gamma=100_Oblate/FD-images/'
    #         plt.savefig(filename + 'TEST_' + str(i) + '.png',dpi=600)
    
    ###Final Step
    vel +=0.5*TF*dt

    mechErrorNorm = np.sum(np.linalg.norm(TF, axis=1))
    if (mechErrorNorm < tolerance) or (mechErrorNorm > maxError):
        break
    if i%outfrequency==0:
        print('timestep =',i+1)
        print('dVolume/Volume_t =',Volume_current-Volume_t,'/',Volume_t)
        print('dArea/Area_t =',Area_current-Area_t,'/',Area_t)
        print('mechErrorNorm =',mechErrorNorm)
        EnergyArea.append(Energy_area(v,f,Area_t))
        EnergyVolume.append(Energy_volume(v,f,Volume_t))
        EnergyBending.append(Energy_bending(v,f,H0))
        KineticEnergy.append(Kinetic_energy(v,f,vel))
        print('KE = ',KineticEnergy[-1])
        PotentialEnergy = np.array(EnergyArea) + np.array(EnergyVolume) + np.array(EnergyBending)
        print('PE = ',PotentialEnergy[-1])
        totalEnergy = np.array(KineticEnergy) + PotentialEnergy
        mp.jupyter()
        p=mp.plot(v,f,shading={"wireframe":True, "wire_color": "black", #Wireframerendering
                               "width": 300, "height": 300},return_plot=True)
        name = 'test'+str(i)
       
        #print(vel_new)
        #df.close()

timestep = 1
dVolume/Volume_t = 1.5120876734395474 / 2.512
dArea/Area_t = -0.08312748230450495 / 12.56
mechErrorNorm = 122.18253280782031
KE =  0.00019042680407665223
PE =  9.376563568872728


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0013659…

timestep = 101
dVolume/Volume_t = 1.1877907093364892 / 2.512
dArea/Area_t = -0.4385712486863209 / 12.56
mechErrorNorm = 60.88733874726914
KE =  0.02760302767012411
PE =  6.206172822264215


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0016904…

timestep = 201
dVolume/Volume_t = 0.8223100598605178 / 2.512
dArea/Area_t = -0.2657914591126165 / 12.56
mechErrorNorm = 23.29537993222267
KE =  0.030610935946651895
PE =  3.149867673855214


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0015497…

In [ ]:
time=np.linspace(0,iterations+1,len(totalEnergy))
plt.plot(time,totalEnergy,'o-', color='red', label='total Energy')
plt.plot(time,EnergyBending,'o-', label='Bending Energy')
plt.plot(time,EnergyVolume,'v-', label='Volume Energy')
plt.plot(time,EnergyArea,'o-', label='Area Energy')
plt.plot(time,PotentialEnergy,'v-',label='Potential Energy')
plt.plot(time,KineticEnergy,'v-', label='Kinetic Energy',color='k')
#plt.axis([0, 800, 0, 2])
plt.legend(loc='upper left')
plt.title('time vs Energy')
plt.show()